<a href="https://colab.research.google.com/github/ritashreedey/Deep-Learning-Project/blob/main/Customer%20Churn_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.11.0


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
X = df.iloc[:,3:13]
y = df.iloc[:,13]

In [ ]:
##Feature Engineering
geography = pd.get_dummies(X['Geography'] , drop_first = True)
gender = pd.get_dummies(X['Gender'] , drop_first = True)

In [ ]:
X = X.drop(['Geography','Gender'] , axis = 1)
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,42,2,0.00,1,1,1,101348.88
1,608,41,1,83807.86,1,0,1,112542.58
2,502,42,8,159660.80,3,1,0,113931.57
3,699,39,1,0.00,2,0,0,93826.63
4,850,43,2,125510.82,1,1,1,79084.10


In [ ]:
X = pd.concat([X , geography ,gender] , axis = 1)
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.2 ,random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU , PReLU , ELU , ReLU
from tensorflow.keras.layers import Dropout


In [ ]:
classifier = Sequential()

In [ ]:
classifier.add(Dense(units = 11 , activation = 'relu'))    #input layer
classifier.add(Dense(units = 7, activation = 'relu'))      #1st hidden layer
classifier.add(Dense(units = 6, activation = 'relu'))      #2nd hidden layer
classifier.add(Dense(1 , activation = 'sigmoid'))          #output layer

In [ ]:
classifier.compile(optimizer = 'adam' , loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# Early Stopping

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=20,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [ ]:
model_history = classifier.fit(X_train , y_train ,validation_split = 0.33 , batch_size = 10 , epochs =1000 , callbacks = early_stopping)

Epoch 1/1000
536/536 [==============================] - 2s 5ms/step - loss: 0.3067 - accuracy: 0.8746 - val_loss: 0.3827 - val_accuracy: 0.8459
Epoch 2/1000
536/536 [==============================] - 2s 4ms/step - loss: 0.3058 - accuracy: 0.8753 - val_loss: 0.3810 - val_accuracy: 0.8459
Epoch 3/1000
536/536 [==============================] - 2s 5ms/step - loss: 0.3061 - accuracy: 0.8755 - val_loss: 0.3844 - val_accuracy: 0.8459
Epoch 4/1000
536/536 [==============================] - 2s 5ms/step - loss: 0.3051 - accuracy: 0.8740 - val_loss: 0.3830 - val_accuracy: 0.8459
Epoch 5/1000
536/536 [==============================] - 2s 4ms/step - loss: 0.3071 - accuracy: 0.8755 - val_loss: 0.3807 - val_accuracy: 0.8429
Epoch 6/1000
536/536 [==============================] - 2s 4ms/step - loss: 0.3055 - accuracy: 0.8755 - val_loss: 0.3837 - val_accuracy: 0.8432
Epoch 7/1000
536/536 [==============================] - 2s 4ms/step - loss: 0.3063 - accuracy: 0.8720 - val_loss: 0.3848 - val_accuracy:

In [ ]:
model_history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [ ]:
# Predicting

y_pred = classifier.predict(X_test)
y_pred = (y_pred >= 0.5)

63/63 [==============================] - 0s 1ms/step


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
cm = confusion_matrix(y_test , y_pred)
score = accuracy_score(y_test , y_pred)

In [ ]:
cm

array([[1517,   78],
       [ 214,  191]])

In [ ]:
score

0.854